# <div style='color:white;background: #005792;text-align: center;padding: 15px 0'>Recommandations - Modélisation d'un modèle machine learning basé sur du Filtrage Collaboratif </div>

## Participants
* Samantha
* Rachelle
* Andrew

### Installation des librairies

In [ ]:
#!pip install scikit-learn
#!pip install tabulate

### Importation des librairies

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

### Chargement des fichiers 

In [2]:
source_dir= '/home/dstrec/dstrec/010_data/002_merged'

file_final = f"{source_dir}/dstrec.csv"

In [3]:
df = pd.read_csv(file_final)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 164108 entries, 0 to 164107
Data columns (total 39 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   imdbId         164108 non-null  object 
 1   title          164108 non-null  object 
 2   averageRating  164108 non-null  float64
 3   numVotes       164108 non-null  float64
 4   director       164108 non-null  object 
 5   actor_actress  164108 non-null  object 
 6   producer       164108 non-null  object 
 7   Action         164108 non-null  int64  
 8   Adult          164108 non-null  int64  
 9   Adventure      164108 non-null  int64  
 10  Animation      164108 non-null  int64  
 11  Biography      164108 non-null  int64  
 12  Comedy         164108 non-null  int64  
 13  Crime          164108 non-null  int64  
 14  Documentary    164108 non-null  int64  
 15  Drama          164108 non-null  int64  
 16  Family         164108 non-null  int64  
 17  Fantasy        164108 non-nul

In [4]:
df.head()

,imdbId,title,averageRating,numVotes,director,actor_actress,producer,Action,Adult,Adventure,...,Romance,Sci,Show,Sport,TV,Talk,Thriller,War,Western,forAdult
0,tt0000009,Miss Jerry,5.4,212.0,Alexander Black,William Courtenay/Blanche Bayliss,Alexander Black,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,tt0000574,The Story of the Kelly Gang,6.0,900.0,Charles Tait,Godfrey Cass/Bella Cola,W.A. Gibson,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2,tt0001370,Rainha Depois de Morta Inês de Castro,5.2,26.0,Carlos Santos,Eduardo Brazão/Amelia Vieira,Júlio Costa,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0001790,"Les Misérables, Part 1: Jean Valjean",6.0,52.0,Albert Capellani,Jean Angelo/Maria Fromet,Pierre Decourcelle,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0001911,Nell Gwynne,3.6,26.0,Raymond Longford,Walter Bastin/Agnes Keogh,George Musgrove,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Implémentation du modèle

In [5]:
#Préparation des données
df['desc']=''
for i in df[df.columns[2:38]].columns.values:
    if df.select_dtypes !=('object'):
        df['desc']=df['desc']+" "+df[i].apply(str)
    else:
        df['desc']=df['desc']+" "+df[i]

In [6]:
# Créer un TfidfVectorizer et supprimer les mots vides
tfidf = TfidfVectorizer(stop_words='english')

#df['desc']=df['director']+df['actor_actress']+df['producer']

# Adapter et transformer les données en une matrice tfidf
matrice_tfidf = tfidf.fit_transform(df['desc'])

In [7]:
# Fonction pour calculer la similarité et obtenir les recommandations
def get_recommendations1(title, matrice_tfidf, df):
    if title not in df['title'].values:
        return "Ce film n'est pas dans la base de données."

    # Trouver l'index du film
    idx = df[df['title'] == title].index[0]
    
    # Calculer la similarité entre le film choisi et tous les autres
    cosine_sim = cosine_similarity(matrice_tfidf[idx:idx+1], matrice_tfidf).flatten()
    
    # Obtenir les indices des films les plus similaires
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]  # Top 10, excluant le film lui-même
    
    # Récupérer les titres des films recommandés
    movie_indices = [i[0] for i in sim_scores]
    recommanded_title=df['title'].iloc[movie_indices]
    score=[i[1] for i in sim_scores]

    return list(zip(movie_indices,recommanded_title,score))

### Exemples de recommandations

In [8]:
# Test de la fonction
get_recommendations1("Star Trek", matrice_tfidf, df)

[(97378, 'Star Trek Into Darkness', 0.806958309354247),
 (125577, 'Star Trek Beyond', 0.48176510642288994),
 (123732, 'Star Wars: Episode VII - The Force Awakens', 0.4100682868941919),
 (124099,
  'Star Wars: Episode IX - The Rise of Skywalker',
  0.39525436584129925),
 (106415, 'Super 8', 0.3904688088534379),
 (98571, 'Marisol', 0.374631593334762),
 (39233, 'The Analyst', 0.3524722975109345),
 (129760, 'Mining for Ruby', 0.30039055705871287),
 (157767, 'Fame-ish', 0.25918987130852256),
 (17490, 'Cargo of Love', 0.23467792053521402)]

In [9]:
# Test de la fonction
get_recommendations1("Rogue One: A Star Wars Story", matrice_tfidf, df)

[(123302, 'Beneath a Neon Tide', 0.2661550196292582),
 (132249, 'Six Hot Chicks in a Warehouse', 0.24002629250513569),
 (32378, 'A Rage in Harlem', 0.2368127139873928),
 (65917, 'First Daughter', 0.23364934609586221),
 (60928, 'The Collaboration', 0.23069579649608107),
 (145146, 'How It Ends', 0.22858667519893477),
 (124228, 'Arrival', 0.22845485699484844),
 (73505, 'The Last King of Scotland', 0.22653381144674284),
 (69554, 'Even Money', 0.2256716447590994),
 (76794, 'Footsteps', 0.22556338624448158)]

In [10]:
# Test de la fonction
get_recommendations1("Star Wars: Episode VII - The Force Awakens", matrice_tfidf, df)

[(124099,
  'Star Wars: Episode IX - The Rise of Skywalker',
  0.42656940272432303),
 (106415, 'Super 8', 0.4214046977585216),
 (77273, 'Star Trek', 0.4100682868941919),
 (97378, 'Star Trek Into Darkness', 0.4100682868941919),
 (98571, 'Marisol', 0.4043127383813279),
 (86228, 'Morning Glory', 0.4017898701519004),
 (39233, 'The Analyst', 0.3803978157359017),
 (17490, 'Cargo of Love', 0.2532708783170953),
 (43171, 'Love Secrets', 0.25262055374169134),
 (21494, 'Crime and Passion', 0.24325819296817458)]

In [11]:
# Test de la fonction
get_recommendations1("Serenity", matrice_tfidf, df)

[(78530, 'The Avengers', 0.2565186128817699),
 (122521, 'Avengers: Age of Ultron', 0.2565186128817699),
 (153266, 'PRND', 0.2524096473736803),
 (91321, 'The Cabin in the Woods', 0.25116705002599987),
 (69944, 'Twelve Steps Outside', 0.20787575009249323),
 (88798, 'Peacock', 0.20392912587794995),
 (27826, '3:15 the Moment of Truth', 0.20297840176665286),
 (40263, 'Rushmore', 0.1994395597936004),
 (51715, 'Unbreakable', 0.19298898768575024),
 (79915, 'The Happening', 0.18903115403251422)]

In [12]:
# Test de la fonction
get_recommendations1("Jumanji", matrice_tfidf, df)

[(39193, 'Small Soldiers', 0.2804271116113367),
 (24609, 'The Offenders', 0.2642108948504122),
 (34112, 'Mrs. Doubtfire', 0.25324821028338823),
 (125443, 'Midnight Special', 0.24142610552070792),
 (69391, 'Breach', 0.2388309190633842),
 (48447, 'Get Over It', 0.23809510350045862),
 (44204, 'Luckytown', 0.23654243064592212),
 (32930, "The Gun in Betty Lou's Handbag", 0.23187818175521002),
 (34982,
  'Interview with the Vampire: The Vampire Chronicles',
  0.23171384052691052),
 (50363, 'Bring It On', 0.23108843023035075)]

In [13]:
get_recommendations1("Toy Story", matrice_tfidf, df)

[(113410, 'Toy Story 4', 0.3884118168482989),
 (38502, 'Toy Story 2', 0.38095017527799824),
 (104575, 'Larry Crowne', 0.3606475862722374),
 (62479, 'Cars', 0.30561086099999335),
 (74692, "Charlie Wilson's War", 0.298886837011888),
 (44387, 'Cast Away', 0.29431141490213497),
 (105623, 'S&M Lawn Care', 0.2931187309559537),
 (90296, 'The Stanton Family Grave Robbery', 0.2917249924068664),
 (37540, 'That Thing You Do!', 0.2859922736949554),
 (155737, 'A Man Called Otto', 0.28276691569078494)]

## <div style='background: #005792;text-align: center;padding: 15px 0'> <a style= 'color:white;' >Essais de modèles non concluants</a></div>

### Modèle (KO)

In [24]:
#1ère tentative en utilisant de modèle collaboratif utilisant la metrique de cosimilarité et la metrique euclidienne
#Ce modèle necessitait trop de mémoire de calcul

In [25]:
#Préparation des données
#df['desc']=''
#for i in df[df.columns[4:38]].columns.values:
#    if df.select_dtypes !=('object'):
#        df['desc']=df['desc']+" "+df[i].apply(str)
#    else:
#        df['desc']=df['desc']+" "+df[i]

In [26]:
# Créer un TfidfVectorizer et supprimer les mots vides
#tfidf = TfidfVectorizer(stop_words='english')

#df['desc']=df['director']+df['actor_actress']+df['producer']

# Adapter et transformer les données en une matrice tfidf
#matrice_tfidf = tfidf.fit_transform(df['title'])

In [27]:
#indices = pd.Series(range(0,len(df)), index=df['title'])

In [28]:
#sim_cosinus = cosine_similarity(matrice_tfidf, matrice_tfidf)

In [29]:
# On calcule la similarité euclidienne
#sim_euclidienne = 1 / (1 + euclidean_distances(matrice_tfidf[:1000]))

#print(sim_euclidienne.shape)
#print(type(sim_euclidienne))

In [30]:
#from tabulate import tabulate
#def recommandations(titre, mat_sim, num_recommendations = 10):
    # On récupère l'indice associé au titre qui servira à identifier le livre dans la matrice de similarité
    #idx = indices[titre]

    # On obtient les scores de similarité de tous les films et on les garde les tuples d'indice du film 
    # et score dans une liste
    #scores_similarite = list(enumerate(mat_sim[idx]))

    # On trie les livres en fonction des scores de similarité
    #scores_similarite = sorted(scores_similarite, key=lambda x: x[1], reverse=True)

    # Obtenir les scores des 10 livres les plus similaires
    #top_similair = scores_similarite[1:num_recommendations+1]

    # Obtenir les indices des livres
    #res = [(indices.index[idx], score) for idx, score in top_similair]

    # Renvoyer les titres des livres les plus similaires
    #return tabulate(res, headers=["Titre", "Score de similarité"], tablefmt="pretty")

In [31]:
#print("Recommandations pour 'Miss Jerry' similarité cosinus: \n",
#      recommandations('Miss Jerry', sim_cosinus))

#print("\n Recommandations pour 'Miss Jerry' similarité euclidienne: \n",
#      recommandations('Miss Jerry', sim_euclidienne))

#print("\n Recommandations pour 'The Story of the Kelly Gang' similarité cosinus: \n",
#      recommandations('The Story of the Kelly Gang', sim_cosinus))

#print("\n Recommandations pour The Story of the Kelly Gang' similarité euclidienne: \n",
#      recommandations('The Story of the Kelly Gang', sim_euclidienne))

### Variante avec la metrique euclidienne (recommandations similaires)

In [ ]:
#ce calcul apporte des recommandations identiques à la recommandation avec la metric cosine

In [ ]:
#Préparation des données
#df['desc']=''
#for i in df[df.columns[4:38]].columns.values:
#    if df.select_dtypes !=('object'):
#        df['desc']=df['desc']+" "+df[i].apply(str)
#    else:
#        df['desc']=df['desc']+" "+df[i]

In [ ]:
# Créer un TfidfVectorizer et supprimer les mots vides
#tfidf = TfidfVectorizer(stop_words='english')

#df['desc']=df['director']+df['actor_actress']+df['producer']

# Adapter et transformer les données en une matrice tfidf
#matrice_tfidf = tfidf.fit_transform(df['title'])

In [ ]:
#indices = pd.Series(range(0,len(df)), index=df['title'])

In [ ]:
#sim_cosinus = cosine_similarity(matrice_tfidf, matrice_tfidf)

In [ ]:
# Fonction pour calculer la similarité et obtenir les recommandations
#def get_recommendations2(title, matrice_tfidf, df):
    #if title not in df['title'].values:
        #return "Ce film n'est pas dans la base de données."

    # Trouver l'index du film
    #idx = df[df['title'] == title].index[0]
    
    # Calculer la similarité entre le film choisi et tous les autres
    #cosine_sim = cosine_similarity(matrice_tfidf[idx:idx+1], matrice_tfidf).flatten()
    #sim_euclidienne = 1/(1+euclidean_distances(matrice_tfidf[idx:idx+1], matrice_tfidf)).flatten()
    
    # Obtenir les indices des films les plus similaires
    #sim_scores = list(enumerate(sim_euclidienne))
    #sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #sim_scores = sim_scores[1:11]  # Top 10, excluant le film lui-même
    
    # Récupérer les titres des films recommandés
    #movie_indices = [i[0] for i in sim_scores]
    #recommanded_title=df['title'].iloc[movie_indices]
    #score=[i[1] for i in sim_scores]
    #return df['title'].iloc[movie_indices]
    #return list(zip(movie_indices, recommanded_title,score))

In [ ]:
# Test de la fonction
#get_recommendations2("Star Trek", matrice_tfidf, df)

### Autre essai de modèle -> resultats non concluants

In [ ]:
#!pip install surprise

In [ ]:
#from surprise import Reader
#from surprise import Dataset

In [ ]:
#df['desc']=''
#for i in df[df.columns[7:38]].columns.values:
    #print(i)
    #if df.select_dtypes !=('object'):
        #df['desc']=df['desc']+df[i].apply(str)
    #else:
        #df['desc']=df['desc']+df[i]

#print(df['desc'])
#print(df['title'])

In [ ]:
#reader = Reader(rating_scale=(0, 5))

#df['ScorePondere']=df['numVotes']*df['averageRating']

#df_surprise = Dataset.load_from_df(df[["desc", "title", "ScorePondere"]], reader=reader)

In [ ]:
#from surprise import NormalPredictor
#from surprise.model_selection import cross_validate

#normpred = NormalPredictor()

#cross_validate(normpred, df_surprise, measures=['RMSE', 'MAE'], cv=5, verbose=True)

In [ ]:
#type_movie="0000000010000000000000000000000"

In [ ]:
# Construire le jeu d'entraînement complet à partir du DataFrame df_surprise
#train_set = df_surprise.build_full_trainset()

# Initialiser une liste vide pour stocker les paires (utilisateur, tittle) pour le jeu "anti-testset"
#anti_testset = []

# Convertir l'ID de l'utilisateur externe en l'ID interne utilisé par Surprise
#targetUser = train_set.to_inner_uid(type_movie)

# Obtenir la valeur de remplissage à utiliser (moyenne globale des notes du jeu d'entraînement)
#moyenne = train_set.global_mean

# Obtenir les évaluations de l'utilisateur cible pour les films
#user_note = train_set.ur[targetUser]

# Extraire la liste des livres notés par l'utilisateur
#user_movie = [item for (item,_) in (user_note)]

# Obtenir toutes les notations du jeu d'entraînement
#ratings = train_set.all_ratings()

# Boucle sur tous les items du jeu d'entraînement
#for movie in train_set.all_items():
    # Si l'item n'a pas été noté par l'utilisateur
    #if movie not in user_movie:
        # Ajouter la paire (utilisateur, movie, valeur de remplissage) à la liste "anti-testset"
        #anti_testset.append((type_movie, train_set.to_raw_iid(movie), moyenne))

In [ ]:
# Effectuer les prédictions sur l'ensemble "anti-testset_user" en utilisant le modèle (non spécifié dans le code)
#predictions = normpred.test(anti_testset)  

# Convertir les prédictions en un DataFrame pandas
#predictions = pd.DataFrame(predictions)

# Trier les prédictions par la colonne 'est' (estimation) en ordre décroissant
#predictions.sort_values(by=['est'], inplace=True, ascending=False)

# Afficher les 10 meilleures prédictions
#predictions.head(10)